# Results Formatting

This notebook was used for collecting and formatting the results of the experiments, for the evaluation section of the paper.

### Imports

In [1]:
import pandas as pd
import os
import src.Common.Utils.Config.ConfigHelper as ConfigHelper
import shutil
import src.Common.EpisodeReplay.EpisodeReplay as EpisodeReplay
from tqdm import tqdm
import pyperclip as pc
import scipy.stats as stats
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Define Constants

In [2]:
RunGroup = "13"
EnvNames = ["FrozenLake"]
BehaviouralTypes = ["Human", "Curated", "HighScore"]
BehaviouralTypesToReview = ["Human", "Curated"]
AgentTypes = ["HardCoded", "ML", "Random", "Human"]

# manual Review of the results config
MaxChoicesPerAgent = 5
MaxReplaysPerChoice = 5


In [131]:
# copy demos of each behavioural type to the run group folder
def CopyDemos(envName, runGroup):
	fromPath = os.path.join("Data", envName, "dev", "replays", "Human")
	toPath = os.path.join("Data", envName, runGroup, "replays", "Human")

	if os.path.exists(toPath):
		shutil.rmtree(toPath)
	shutil.copytree(fromPath, toPath)

	print("Copied Human Demos for: " + envName)
	return

def AddDemoIdsToBehaviour(envName, runGroup, behaviourType):
	# load the results
	statsPath = os.path.join("Data", envName, runGroup, "replays", "Human", "stats.tsv")
	stats = pd.read_csv(statsPath, sep="\t")

	lowerBehaviourType = behaviourType.lower()

	stats["Behaviour"] = stats["metricName"].apply(lambda x: x.split("_")[-2])

	if lowerBehaviourType != "highscore":
		stats = stats[stats["Behaviour"] == lowerBehaviourType]

	episodeIds = stats["EpisodeId"].unique().tolist()

	metricName = "Human_" + behaviourType

	# load the json with the episode Ids of the behavioural type
	episodeIdsPath = os.path.join("Data", envName, runGroup, f"{behaviourType}_Episodes.json")
	
	episodeIdsJson = ConfigHelper.LoadConfig(episodeIdsPath)
	episodeIdsJson[metricName] = episodeIds

	ConfigHelper.SaveConfig(episodeIdsJson, episodeIdsPath)

	print(f"Added {len(episodeIds)} {behaviourType} episodes to {envName}")
	return

for envName in EnvNames:
	CopyDemos(envName, RunGroup)

	for behaviourType in BehaviouralTypes:
		AddDemoIdsToBehaviour(envName, RunGroup, behaviourType)


Copied Human Demos for: FrozenLake
Added 59 Human episodes to FrozenLake
Added 23 Curated episodes to FrozenLake
Added 82 HighScore episodes to FrozenLake


## Collect Replays For Manual Reviewed

In [4]:
def LoadReplay(envName, runGroup, agentType, episodeId):
	
	path = os.path.join("Data", envName, runGroup, "replays", agentType, episodeId)
	try:
		replay = EpisodeReplay.EpisodeReplay.LoadFromFolder(path)
		return replay
	except:
		return None

def CollectReplaysToReview(envName, runGroup, behaviourType):
	episodeIdsPath = os.path.join("Data", envName, runGroup, f"{behaviourType}_Episodes.json")
	replays =  ConfigHelper.LoadConfig(episodeIdsPath)

	columns = ["AgentId", "Predicted", "AgentType"]
	columns += [f"Replay_{i}" for i in range(MaxReplaysPerChoice)]

	replaysToReview = pd.DataFrame(columns=columns)

	for agentId, episodeIds in replays.items():
		
		agentType = agentId.split("_")[0]
		ids = []

		for i in range(len(episodeIds)):
			episodeId = episodeIds[i]

			replay = LoadReplay(envName, runGroup, agentType, episodeId)
			if replay is None:
				continue

			ids.append(episodeId)

			if len(ids) >= MaxReplaysPerChoice or i == len(episodeIds) - 1:
				row = {}
				row["AgentId"] = [agentId]
				row["Predicted"] = [None]
				row["AgentType"] = [agentType]
				for i, id in enumerate(ids):
					row[f"Replay_{i}"] = [id]

				replaysToReview = pd.concat([replaysToReview, pd.DataFrame(row)], ignore_index=True)
				ids = []

	return replaysToReview
	
for envName in EnvNames:
	for behaviourType in BehaviouralTypesToReview:

		replaysToReview = CollectReplaysToReview(envName, RunGroup, behaviourType)
		replaysToReview = replaysToReview.sample(frac=1)
		replaysToReviewPath = os.path.join("Data", envName, RunGroup, f"ReplaysToReview_{behaviourType}.json")
		replaysToReview.to_json(replaysToReviewPath, orient="records", indent=4)

		print(f"Collected {len(replaysToReview)} replays to review for {behaviourType} in {envName}")


## Formate the results of the review

In [3]:
def CalculateBinaryCI(positives, totalCount, confidenceLevel=0.95):
	# Calculate the sample proportion (p)
	p = positives / totalCount
	
	# Calculate the standard error
	se = math.sqrt((p * (1 - p)) / totalCount)
	
	# Calculate the Z-score for the desired confidence level
	z = stats.norm.ppf(1 - (1 - confidenceLevel) / 2)
	
	# Calculate the margin of error
	margin_of_error = z * se
	
	# Calculate the lower and upper bounds of the confidence interval
	lower_bound = p - margin_of_error
	upper_bound = p + margin_of_error
	
	return lower_bound, upper_bound

In [26]:
def CalStandardError(positives, totalCount):
	p = positives / totalCount
	return math.sqrt((p * (1 - p)) / totalCount)

In [31]:
def LoadReplaysToReview(envName, runGroup, behaviourType):
	replaysToReviewPath = os.path.join("Data", envName, runGroup, f"ReplaysToReview_{behaviourType}.json")
	replaysToReview = pd.read_json(replaysToReviewPath)

	grouped = replaysToReview.groupby("AgentType").aggregate({"Predicted": ["count", "sum"]})


	grouped["Percent"] = grouped["Predicted"]["sum"] / grouped["Predicted"]["count"]
	grouped["Norm_Percent"] = grouped["Percent"] / grouped["Percent"]["Human"]

	# calculate the confidence intervals
	grouped["Error"] = grouped.apply(lambda x: CalStandardError(x["Predicted"]["sum"], x["Predicted"]["count"]), axis=1)
	return grouped

for envName in EnvNames:
	for behaviourType in BehaviouralTypesToReview:
		grouped = LoadReplaysToReview(envName, RunGroup, behaviourType)
		print(f"{envName} - {behaviourType}")
		display(grouped)

FrozenLake - Human


Predicted     Percent Norm_Percent     Error
              count sum                               
AgentType                                             
HardCoded        20   1    0.05     0.066667  0.048734
Human            12   9    0.75     1.000000  0.125000
ML               40  34    0.85     1.133333  0.056458
Random           20   0    0.00     0.000000  0.000000

FrozenLake - Curated


C:\Users\louie\AppData\Local\Temp\ipykernel_23480\2844687134.py:2: RuntimeWarning: invalid value encountered in double_scalars
  p = positives / totalCount


Predicted      Percent Norm_Percent Error
              count  sum                           
AgentType                                          
HardCoded         0  0.0     NaN          NaN   NaN
Human             0  0.0     NaN          NaN   NaN
ML                0  0.0     NaN          NaN   NaN
Random            0  0.0     NaN          NaN   NaN

## Formating tables and graphs

In [4]:
def CollectEvalIds(runGroup, envNames, behaviouralTypes):
	columns = ["EnvName", "AgentId", "AgentType", "EpisodeId", "Behaviour"]
	df = pd.DataFrame(columns=columns)

	for envName in envNames:
		for behaviourType in behaviouralTypes:
			episodeIdsPath = os.path.join("Data", envName, runGroup, f"{behaviourType}_Episodes.json")
			replays =  ConfigHelper.LoadConfig(episodeIdsPath)


			for agentId, episodeIds in replays.items():
				
				agentType = agentId.split("_")[0]

				for i in range(len(episodeIds)):
					episodeId = episodeIds[i]

					row = {}
					row["EnvName"] = [envName]
					row["AgentId"] = [agentId]
					row["AgentType"] = [agentType]
					row["Behaviour"] = [behaviourType]
					row["EpisodeId"] = [episodeId]
					df = pd.concat([df, pd.DataFrame(row)], ignore_index=True)

	return df

evalIds = CollectEvalIds(RunGroup, EnvNames, BehaviouralTypes)

NameError: name 'RunGroup' is not defined

In [ ]:
def CleanStats(df):
	prefixesToDrop = ["LearnerConfig", "ModelConfigs", "DataTables"]
	columnsToDrop = [col for col in df.columns if col.startswith(tuple(prefixesToDrop))]
	df = df.drop(columns=columnsToDrop)

	# add duration column
	df["Duration"] = (df["EndTime"] - df["StartTime"])  / 1e9
	return df

def CombinedStats(runGroup, envNames, agentTypes):
	combinedStats = None

	for envName in envNames:
		for agentType in agentTypes:
			statsPath = os.path.join("Data", envName, runGroup, "replays", agentType, "stats.tsv")
			stats = pd.read_csv(statsPath, sep="\t")
			stats = CleanStats(stats)

			combinedStats = pd.concat([combinedStats, stats], ignore_index=True)
	return combinedStats

stats = CombinedStats(RunGroup, EnvNames, AgentTypes)
evalDf = pd.merge(evalIds, stats, on=["EpisodeId"], how="left")

# drop rows with nan values
evalDf = evalDf.dropna()

In [ ]:
def ConvertToLatex(df):
	# Get column names
	columns = df.columns.tolist()

	headerCode = "\hline\n"
	headerCode += "\t\multicolumn{1}{|c|}{\\textbf{"
	headerCode += "}} &\n\t\multicolumn{1}{c|}{\\textbf{".join(columns)

	headerCode += "}} \\\\\n\hline\n"


	# Generate LaTeX table code
	latex_code = "\\begin{longtable}{|" + "c|" * len(columns) + "}\n"

	# add caption and label
	latex_code += "\\caption{Insert Caption Here.}\n"
	latex_code += "\\label{tab:InsertLabelHere} \\\\\n"

	latex_code += headerCode
	latex_code += "\endfirsthead\n\n"

	latex_code += "\multicolumn{" + str(len(columns)) + "}{c}%\n"
	latex_code += "{{\\bfseries \\tablename\\ \\thetable{} -- continued from previous page}} \\\\\n"
	latex_code += headerCode
	latex_code += "\endhead\n\n"

	latex_code += "\hline \multicolumn{" + str(len(columns)) + "}{|c|}{{Continued on next page}} \\\\ \hline\n\n"
	latex_code += "\endfoot\n"

	latex_code += "\hline\n"
	latex_code += "\endlastfoot\n"

	latex_code += "\n"


	# Add data rows
	for index, row in df.iterrows():
		values = row.tolist()
		latex_code += "\t" + " & ".join(str(value) for value in values) + " \\\\\n"
	latex_code += "\\hline\n"
	# Complete LaTeX table code
	latex_code += "\\end{longtable}"

	return latex_code

In [ ]:
aggregateSettings = {}
aggregateSettings["EpisodeId"] = "count"
aggregateSettings["Duration"] = ["mean"]
aggregateSettings["EpisodeTotalReward"] = ["mean", "std"]
aggregateSettings["EpisodeTotalCuratedReward"] = ["mean", "std"]

evalDf.groupby(["EnvName", "Behaviour", "AgentId"]).aggregate(aggregateSettings)

EpisodeId  Duration  \
                                                         count      mean   
EnvName    Behaviour AgentId                                               
FrozenLake Curated   HardCoded_D_1_RT_True_Curated         100  0.039966   
                     ML_D_10_RT_False_Curated              100  0.340874   
                     ML_D_10_RT_True_Curated               100  0.169350   
                     Random_D_1_RT_True_Curated            112  2.317554   
           HighScore HardCoded_D_1_RT_True_HighScore      1000  0.020397   
                     ML_D_10_RT_False_HighScore           1000  0.380918   
                     ML_D_10_RT_True_HighScore            1000  0.138895   
                     Random_D_1_RT_True_HighScore         1008  2.385675   
           Human     HardCoded_D_1_RT_True_Human           100  0.014433   
                     ML_D_10_RT_False_Human                100  0.884688   
                     ML_D_10_RT_True_Human                 100  0.345229   
                     Random_D_1_RT_True_Human              116  1.755059   

                                                     EpisodeTotalReward  \
                                                                   mean   
EnvName    Behaviour AgentId                                              
FrozenLake Curated   HardCoded_D_1_RT_True_Curated             1.000000   
                     ML_D_10_RT_False_Curated                  1.000000   
                     ML_D_10_RT_True_Curated                   1.000000   
                     Random_D_1_RT_True_Curated                0.044643   
           HighScore HardCoded_D_1_RT_True_HighScore           1.000000   
                     ML_D_10_RT_False_HighScore                1.000000   
                     ML_D_10_RT_True_HighScore                 1.000000   
                     Random_D_1_RT_True_HighScore              0.038690   
           Human     HardCoded_D_1_RT_True_Human               1.000000   
                     ML_D_10_RT_False_Human                    1.000000   
                     ML_D_10_RT_True_Human                     1.000000   
                     Random_D_1_RT_True_Human                  0.025862   

                                                                \
                                                           std   
EnvName    Behaviour AgentId                                     
FrozenLake Curated   HardCoded_D_1_RT_True_Curated    0.000000   
                     ML_D_10_RT_False_Curated         0.000000   
                     ML_D_10_RT_True_Curated          0.000000   
                     Random_D_1_RT_True_Curated       0.207447   
           HighScore HardCoded_D_1_RT_True_HighScore  0.000000   
                     ML_D_10_RT_False_HighScore       0.000000   
                     ML_D_10_RT_True_HighScore        0.000000   
                     Random_D_1_RT_True_HighScore     0.192952   
           Human     HardCoded_D_1_RT_True_Human      0.000000   
                     ML_D_10_RT_False_Human           0.000000   
                     ML_D_10_RT_True_Human            0.000000   
                     Random_D_1_RT_True_Human         0.159412   

                                                     EpisodeTotalCuratedReward  \
                                                                          mean   
EnvName    Behaviour AgentId                                                     
FrozenLake Curated   HardCoded_D_1_RT_True_Curated                    1.000000   
                     ML_D_10_RT_False_Curated                         0.000000   
                     ML_D_10_RT_True_Curated                          0.000000   
                     Random_D_1_RT_True_Curated                       0.089286   
           HighScore HardCoded_D_1_RT_True_HighScore                  0.000000   
                     ML_D_10_RT_False_HighScore                       0.000000   
                     ML_D_10_RT_True_HighScore           

In [ ]:


evalDf[evalDf["Behaviour"] == "HighScore"].groupby(["EnvName", "AgentId"])[["EpisodeTotalReward"]].aggregate(aggregateSettings["EpisodeTotalReward"])

EpisodeTotalReward          
                                                         mean       std
EnvName    AgentId                                                     
FrozenLake HardCoded_D_1_RT_True_HighScore            1.00000  0.000000
           ML_D_10_RT_False_HighScore                 1.00000  0.000000
           ML_D_10_RT_True_HighScore                  1.00000  0.000000
           Random_D_1_RT_True_HighScore               0.03869  0.192952

In [ ]:
def CreateEnvAgentTypeTable(evalDf, envNames, agentTypes, metric):

	columns = ["EnvName"]
	columns += agentTypes

	df = pd.DataFrame(columns=columns)

	for envName in envNames:
		row = {}
		row["EnvName"] = [envName]

		for agentType in agentTypes:
			agentTypeDf = evalDf[evalDf["AgentType"] == agentType]
			agentTypeDf = agentTypeDf[agentTypeDf["EnvName"] == envName]

			avg = agentTypeDf[metric].mean()
			error = agentTypeDf[metric].std()
			cell = f"{avg:.2f}"# ±{error:.2f}"
			row[agentType] = [cell]

		df = pd.concat([df, pd.DataFrame(row)], ignore_index=True)

	
	# set the index to be the env name
	df = df.set_index("EnvName")

	text = ConvertToLatex(df)
	pc.copy(text)
	print(text.replace("\n", " "))
	display(df)
	print("Copied to clipboard")

	return

In [ ]:
curatedDf = evalDf[evalDf["Behaviour"] == "Curated"]
humanDf = evalDf[evalDf["Behaviour"] == "Human"]
highScoreDf = evalDf[evalDf["Behaviour"] == "HighScore"]

NameError: name 'evalDf' is not defined

In [ ]:
CreateEnvAgentTypeTable(highScoreDf, EnvNames, AgentTypes, "EpisodeTotalReward")

\begin{longtable}{|c|c|c|c|} \caption{Insert Caption Here.} \label{tab:InsertLabelHere} \\ \hline 	\multicolumn{1}{|c|}{\textbf{HardCoded}} & 	\multicolumn{1}{c|}{\textbf{ML}} & 	\multicolumn{1}{c|}{\textbf{Random}} & 	\multicolumn{1}{c|}{\textbf{Human}} \\ \hline \endfirsthead  \multicolumn{4}{c}% {{\bfseries \tablename\ \thetable{} -- continued from previous page}} \\ \hline 	\multicolumn{1}{|c|}{\textbf{HardCoded}} & 	\multicolumn{1}{c|}{\textbf{ML}} & 	\multicolumn{1}{c|}{\textbf{Random}} & 	\multicolumn{1}{c|}{\textbf{Human}} \\ \hline \endhead  \hline \multicolumn{4}{|c|}{{Continued on next page}} \\ \hline  \endfoot \hline \endlastfoot  	1.00 ±0.00 & 1.00 ±0.00 & 0.04 ±0.19 & nan ±nan \\ \hline \end{longtable}


,HardCoded,ML,Random,Human
EnvName,,,,
FrozenLake,1.00 ±0.00,1.00 ±0.00,0.04 ±0.19,nan ±nan


Copied to clipboard


In [ ]:
CreateEnvAgentTypeTable(curatedDf, EnvNames, AgentTypes, "EpisodeTotalCuratedReward")

\begin{longtable}{|c|c|c|c|} \caption{Insert Caption Here.} \label{tab:InsertLabelHere} \\ \hline 	\multicolumn{1}{|c|}{\textbf{HardCoded}} & 	\multicolumn{1}{c|}{\textbf{ML}} & 	\multicolumn{1}{c|}{\textbf{Random}} & 	\multicolumn{1}{c|}{\textbf{Human}} \\ \hline \endfirsthead  \multicolumn{4}{c}% {{\bfseries \tablename\ \thetable{} -- continued from previous page}} \\ \hline 	\multicolumn{1}{|c|}{\textbf{HardCoded}} & 	\multicolumn{1}{c|}{\textbf{ML}} & 	\multicolumn{1}{c|}{\textbf{Random}} & 	\multicolumn{1}{c|}{\textbf{Human}} \\ \hline \endhead  \hline \multicolumn{4}{|c|}{{Continued on next page}} \\ \hline  \endfoot \hline \endlastfoot  	1.00 ±0.00 & 0.00 ±0.00 & 0.09 ±0.29 & nan ±nan \\ \hline \end{longtable}


,HardCoded,ML,Random,Human
EnvName,,,,
FrozenLake,1.00 ±0.00,0.00 ±0.00,0.09 ±0.29,nan ±nan


Copied to clipboard


In [ ]:
CreateEnvAgentTypeTable(highScoreDf, EnvNames, AgentTypes, "Duration")

\begin{longtable}{|c|c|c|c|} \caption{Insert Caption Here.} \label{tab:InsertLabelHere} \\ \hline 	\multicolumn{1}{|c|}{\textbf{HardCoded}} & 	\multicolumn{1}{c|}{\textbf{ML}} & 	\multicolumn{1}{c|}{\textbf{Random}} & 	\multicolumn{1}{c|}{\textbf{Human}} \\ \hline \endfirsthead  \multicolumn{4}{c}% {{\bfseries \tablename\ \thetable{} -- continued from previous page}} \\ \hline 	\multicolumn{1}{|c|}{\textbf{HardCoded}} & 	\multicolumn{1}{c|}{\textbf{ML}} & 	\multicolumn{1}{c|}{\textbf{Random}} & 	\multicolumn{1}{c|}{\textbf{Human}} \\ \hline \endhead  \hline \multicolumn{4}{|c|}{{Continued on next page}} \\ \hline  \endfoot \hline \endlastfoot  	0.02 ±0.00 & 0.26 ±2.74 & 2.39 ±1.41 & nan ±nan \\ \hline \end{longtable}


,HardCoded,ML,Random,Human
EnvName,,,,
FrozenLake,0.02 ±0.00,0.26 ±2.74,2.39 ±1.41,nan ±nan


Copied to clipboard
